In [1]:
pip install pandas numpy scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp311-cp311-macosx_13_0_x86_64.whl size=1170536 sha256=6c41eb87ff10c4a69dc7ba0b041b17188664b327275e857240d43f57509e5e7d
  Stored in directory: /Users/delinaivanova/Library/Caches/pip/wheels/f4/2b/26/e2a5eae55d3b7688995e66abe7f40473aac6c95ddd8ee174a8
Successfully built scikit-surprise

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import cross_validate

# Load the IMDb dataset (you'll need to download this and adjust the path)
# For demonstration, we're using two CSV files: 'ratings.csv' and 'movies.csv'
ratings_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/ratings.csv')
movies_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/movies.csv')

# Merging movies with ratings (assuming 'movieId' is the common column)
data = pd.merge(ratings_df, movies_df, on='movieId')

# Using Surprise library to handle the data
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

# Use KNN to compute similarities between items
sim_options = {
    'name': 'cosine',
    'user_based': False  # Compute similarities between items
}
algo = KNNBasic(sim_options=sim_options)

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9705  0.9827  0.9768  0.9698  0.9736  0.9747  0.0047  
MAE (testset)     0.7552  0.7661  0.7575  0.7535  0.7563  0.7577  0.0044  
Fit time          8.77    10.92   7.27    7.38    7.34    8.34    1.41    
Test time         7.12    6.77    6.53    6.75    7.06    6.85    0.22    


{'test_rmse': array([0.97049586, 0.98273918, 0.97675487, 0.969788  , 0.97363653]),
 'test_mae': array([0.75523826, 0.76607327, 0.7574745 , 0.75345815, 0.75634031]),
 'fit_time': (8.766852140426636,
  10.917131900787354,
  7.270986795425415,
  7.3827269077301025,
  7.3402650356292725),
 'test_time': (7.122598886489868,
  6.77333402633667,
  6.528990983963013,
  6.749686002731323,
  7.0567450523376465)}

In [9]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise import accuracy
from surprise.model_selection import train_test_split

# Load and merge the IMDb dataset (assuming 'ratings.csv' and 'movies.csv' exist)
ratings_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/ratings.csv')
movies_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/movies.csv')
data = pd.merge(ratings_df, movies_df, on='movieId')

# Preparing the data for the Surprise library
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

# Splitting the dataset into train and test sets
trainset, testset = train_test_split(data, test_size=0.25)

# Using KNN to compute similarities between users
algo = KNNBasic(sim_options={'user_based': True})
algo.fit(trainset)

# Making predictions and evaluating the model
predictions = algo.test(testset)
accuracy.rmse(predictions)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9545


0.9545147879894045

In [10]:
# Using the same initial setup as in the user-based example

# Using KNN to compute similarities between items
algo = KNNBasic(sim_options={'user_based': False})
algo.fit(trainset)

# Making predictions and evaluating the model
predictions = algo.test(testset)
accuracy.rmse(predictions)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9058


0.9057987480277283

In [17]:
userid = 668
movieId = 16
algo.predict(userid,movieId)

Prediction(uid=668, iid=16, r_ui=None, est=3.025, details={'actual_k': 40, 'was_impossible': False})

Prediction(uid=668, iid=16, r_ui=None, est=3.025, details={'actual_k': 40, 'was_impossible': False})

uid: This is the user ID for whom the prediction is made. In this case, uid=668 indicates the prediction is for user 668.

iid: This is the item ID that the prediction is about. Here, iid=16 refers to item 16 (which could be a movie, a product, etc., depending on your dataset).

r_ui: This stands for the real (actual) rating given by the user to the item. r_ui=None means that in this instance, the actual rating is unknown or not provided. This is typical when you're predicting for items that a user has not yet rated.

est: This is the estimated rating that the model predicts the user would give to the item. Here, est=3.025 means the model predicts that user 668 would rate item 16 with a rating of around 3.025.

details: This dictionary provides additional details about the prediction.

actual_k: The number of neighbors that were used to make the prediction. actual_k=40 means that the algorithm used 40 neighbors to calculate the estimated rating.
was_impossible: This flag indicates whether the prediction was possible to make. was_impossible=False means the prediction was successfully made. If it were True, it could mean that there weren’t enough neighbors to make a prediction, or some other issue prevented the algorithm from making a prediction.

In [13]:
data

In [15]:
ratings_df

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523
...,...,...,...,...
105334,668,142488,4.0,1451535844
105335,668,142507,3.5,1451535889
105336,668,143385,4.0,1446388585
105337,668,144976,2.5,1448656898


In [16]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
10324,146684,Cosmic Scrat-tastrophe (2015),Animation|Children|Comedy
10325,146878,Le Grand Restaurant (1966),Comedy
10326,148238,A Very Murray Christmas (2015),Comedy
10327,148626,The Big Short (2015),Drama


EXAMPLE OF SVD (Single Value Decomposition)

In [18]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate


In [19]:
# Initialize the SVD algorithm
svd = SVD()

# Evaluate the performance in terms of RMSE and MAE
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8731  0.8714  0.8638  0.8657  0.8698  0.8687  0.0035  
MAE (testset)     0.6758  0.6704  0.6660  0.6679  0.6711  0.6702  0.0033  
Fit time          1.47    1.24    1.27    1.43    1.28    1.34    0.09    
Test time         0.30    0.11    0.26    0.11    0.24    0.21    0.08    


{'test_rmse': array([0.87313735, 0.87136679, 0.8637571 , 0.8657078 , 0.86975801]),
 'test_mae': array([0.67579672, 0.6703701 , 0.66599019, 0.66786085, 0.67108396]),
 'fit_time': (1.4710519313812256,
  1.238070011138916,
  1.2714629173278809,
  1.4264299869537354,
  1.2757940292358398),
 'test_time': (0.296375036239624,
  0.11462879180908203,
  0.26259303092956543,
  0.11359214782714844,
  0.23913192749023438)}

In [20]:
# Train on the whole dataset
trainset = data.build_full_trainset()
svd.fit(trainset)

# Predict a rating for a specific user and movie
user_id = 668  # Replace with a real user ID
movie_id = 16  # Replace with a real movie ID
predicted_rating = svd.predict(user_id, movie_id).est

print(f"Predicted rating for user {user_id} on movie {movie_id} is: {predicted_rating}")


Predicted rating for user 668 on movie 16 is: 3.2456349107321105


In [24]:
predictions = ratings_df.copy()

In [28]:
predictions

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523
...,...,...,...,...
105334,668,142488,4.0,1451535844
105335,668,142507,3.5,1451535889
105336,668,143385,4.0,1446388585
105337,668,144976,2.5,1448656898


## Applying predictions to full data set based on user and movie

In [29]:
predictions['predicted_rating'] = predictions.apply(lambda x: svd.predict(x['userId'],x['movieId']).est, axis=1)

In [30]:
predictions

,userId,movieId,rating,timestamp,predicted_rating
0,1,16,4.0,1217897793,3.830063
1,1,24,1.5,1217895807,2.424854
2,1,32,4.0,1217896246,4.125193
3,1,47,4.0,1217896556,4.299678
4,1,50,4.0,1217896523,3.936647
...,...,...,...,...,...
105334,668,142488,4.0,1451535844,3.119477
105335,668,142507,3.5,1451535889,2.944587
105336,668,143385,4.0,1446388585,3.085636
105337,668,144976,2.5,1448656898,2.514891
